<a href="https://colab.research.google.com/github/GretaDanesi18/Olive_Detection_Tracking/blob/main/YOLOv5_olive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#clono la repo e installa requisiti

In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
! pip install -r requirements.txt

In [3]:
#librerie

In [93]:
import torch #per usare pytorch
import os
import random 
import shutil
from IPython.display import Image
from PIL import Image, ImageDraw
import numpy as np
import pandas as pd

In [5]:
#controllo utilizzo del device GPU oppure CPU

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
print('Using PyTorch version:', torch.__version__, ' Device:', device)

In [7]:
#clono repo con dataset e file yaml

In [ ]:
!git clone https://github.com/GretaDanesi18/Olive_Detection_Tracking.git

In [10]:
#creo le directory per fare split del dataset

In [9]:
os.makedirs('/content/yolov5/Olive_Detection_Tracking/images/val')
os.makedirs('/content/yolov5/Olive_Detection_Tracking/images/train')
os.makedirs('/content/yolov5/Olive_Detection_Tracking/labels/val')
os.makedirs('/content/yolov5/Olive_Detection_Tracking/labels/train')



In [11]:
#array di appoggio per i dati
#percentuali per lo split

In [ ]:
#array che contiene i file
image = []
label = [] 

#path
all_data_path = '/content/yolov5/Olive_Detection_Tracking/dataset'
train_image_path = '/content/yolov5/Olive_Detection_Tracking/images/train'
val_image_path = '/content/yolov5/Olive_Detection_Tracking/images/val'
train_label_path ='/content/yolov5/Olive_Detection_Tracking/labels/train'
val_label_path = '/content/yolov5/Olive_Detection_Tracking/labels/val'


#percentuale per train, val e test
val_ratio = 0.2
train_ratio = 0.8

#inserisco file negli array corrispondenti
for (dirname, dirs, files) in os.walk(all_data_path):
    for filename in files:
        if filename.endswith('.txt'):
            label.append(filename)
        else:
            image.append(filename)

#numero di immagini per train e per val
num_train = int(len(image)*train_ratio)
num_val = int(len(image)*val_ratio)

print("Immagini per il training: ", num_train)
print("Immagini per validation: ", num_val)
print(image[3])
print(label[3])


In [13]:
#inserisco dati nelle directory per yolov5

In [78]:
#inserisco image e label in train
for i in range(num_train):
   file_image = random.choice(image)
   file_image_name = file_image.split('.')[0]
   file_label =  file_image_name+'.txt'

   #sposto i file nella cartella train
   shutil.copy(os.path.join(all_data_path, file_image), os.path.join(train_image_path, file_image))
   shutil.copy(os.path.join(all_data_path, file_label), os.path.join(train_label_path, file_label))

   #rimuovo i file da arrays
   image.remove(file_image)
   label.remove(file_label)
    

#inserisco image e label in val  
for i in range(num_val):
    file_image =random.choice(image) # get name of random image from origin dir
    file_image_name = file_image.split('.')[0]
    file_label = file_image_name+'.txt'

    #move both files into train dir
    shutil.copy(os.path.join(all_data_path, file_image), os.path.join(val_image_path, file_image))
    shutil.copy(os.path.join(all_data_path, file_label), os.path.join(val_label_path, file_label))
    
    #rimuovo i file da arrays
    image.remove(file_image)
    label.remove(file_label)


In [15]:
#inserisco file yaml in directory data

In [ ]:
source = 'Olive_Detection_Tracking/dataset.yaml'
destination = 'data/dataset.yaml'
shutil.copy(source,destination)

In [ ]:
#Weights & Biases
%pip install -q wandb
import wandb
wandb.login()

In [18]:
#addestramento

In [ ]:
!python train.py --img 416 --batch 16 --epochs 50 --data dataset.yaml --weights yolov5s.pt --cache 

In [20]:
#visualizzazione dati 

In [ ]:
# Start tensorboard
# Launch after you have started training
# logs save in the folder "runs"
%load_ext tensorboard
%tensorboard --logdir runs

In [22]:
#Controllo dove vengono salvati runs/train/.. i risulati e aggiorna directory

In [ ]:
Image.open('runs/train/exp10/train_batch0.jpg')


In [ ]:
Image.open('runs/train/exp10/val_batch0_labels.jpg')

In [ ]:
Image.open('runs/train/exp10/val_batch0_pred.jpg')

In [ ]:

file_csv=pd.read_csv('runs/train/exp10/results.csv')
file_csv.head(-1)


In [ ]:
Image.open('runs/train/exp10/results.png')